## **샘플: 호텔 예약을 위한 다중 AI 에이전트**

오늘날 빠르게 변화하는 세상에서, 출장 계획은 단순히 항공편과 호텔 객실을 예약하는 것 이상을 요구합니다. 이는 높은 수준의 조율과 효율성을 필요로 하며, 이를 달성하는 데 어려움이 따를 수 있습니다. 바로 여기서 다중 AI 에이전트가 등장하여 우리의 여행 관리 방식을 혁신합니다.

지능적인 에이전트 팀이 여러분의 손끝에서 협력하며, 여행의 모든 측면을 정밀하고 간편하게 처리한다고 상상해 보세요. 우리의 첨단 AI 기술을 통해 예약 서비스와 일정 조정을 전문으로 하는 에이전트를 만들어, 매끄럽고 스트레스 없는 여행 경험을 제공합니다.

이것은 기본적인 시나리오입니다. 출장 계획을 세울 때, 항공권 정보와 호텔 정보를 얻기 위해 비즈니스 여행 에이전트와 상담해야 합니다. AI 에이전트를 통해 예약 서비스를 위한 에이전트와 일정 조정을 위한 에이전트를 구축하여 협력하고 지능 수준을 향상시킬 수 있습니다.


# Azure AI Agent Service 초기화 및 **.env** 파일에서 구성 정보 가져오기

### **.env**

.env 파일 생성하기

**.env** 파일에는 Azure AI Agent Service의 연결 문자열, AOAI에서 사용하는 모델, 그리고 관련된 Google API Search 서비스 API, ENDPOINT 등이 포함됩니다.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Azure AI Agent Service 모델 배포 이름"

[**NOTE**] 분당 100,000 토큰 처리 한도와 분당 600 요청 한도를 가진 모델이 필요합니다.

  모델은 Azure AI Foundry의 Model and Endpoint에서 얻을 수 있습니다.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Azure AI Agent Service 프로젝트 연결 문자열"

  프로젝트 연결 문자열은 AI Foundry 포털 화면의 프로젝트 개요에서 확인할 수 있습니다.

- **SERPAPI_SEARCH_API_KEY** = "SERPAPI 검색 API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "SERPAPI 검색 ENDPOINT"

Azure AI Agent Service의 모델 배포 이름과 프로젝트 연결 문자열을 얻으려면 Azure AI Agent Service를 생성해야 합니다. [이 템플릿](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)을 사용하여 직접 생성하는 것을 권장합니다. （***참고:*** Azure AI Agent Service는 현재 제한된 지역에서만 설정 가능합니다. [이 링크](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)를 참고하여 지역을 설정하는 것을 권장합니다.)

Agent는 SERPAPI에 접근해야 합니다. [이 링크](https://serpapi.com/searches)를 통해 등록하는 것을 권장합니다. 등록 후, 고유한 API KEY와 ENDPOINT를 얻을 수 있습니다.


# Azure 로그인

이제 Azure에 로그인해야 합니다. VScode에서 터미널을 열고 `az login` 명령어를 실행하세요.


# 설정

이 노트북을 실행하려면 다음 라이브러리들을 설치해야 합니다. 아래는 필요한 라이브러리와 해당 pip 설치 명령어 목록입니다:

azure-identity: Azure 인증을 위해 사용됩니다.  
requests: HTTP 요청을 수행하기 위해 사용됩니다.  
semantic-kernel: 시맨틱 커널 프레임워크를 위해 사용됩니다 (이 라이브러리가 커스텀 또는 특정 라이브러리라면, 특정 소스나 저장소에서 설치해야 할 수도 있습니다).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# 설명:
import asyncio: 이 코드는 asyncio 모듈을 가져옵니다. asyncio는 Python에서 비동기 프로그래밍을 지원하며, async와 await 구문을 사용하여 동시 실행 코드를 작성할 수 있게 해줍니다.  
from typing import Annotated: 이 코드는 typing 모듈에서 Annotated 타입을 가져옵니다. Annotated는 타입 힌트에 메타데이터를 추가하는 데 사용되며, 검증, 문서화, 도구 지원 등 다양한 목적으로 유용하게 활용될 수 있습니다.  


In [ ]:
import asyncio,os
from typing import Annotated

# 설명:
`from dotenv import load_dotenv`와 `load_dotenv()`를 사용하면 .env 파일에서 설정값과 민감한 정보를 쉽게 관리할 수 있습니다(예: API 키와 데이터베이스 URL). 이를 통해 이러한 정보를 소스 코드와 분리하여 애플리케이션을 더 안전하고 설정하기 쉽게 만들 수 있습니다.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 설명:

Import Statement: from azure.identity.aio import DefaultAzureCredential: 이 코드는 azure.identity.aio 모듈에서 DefaultAzureCredential 클래스를 가져옵니다. 모듈 이름에 포함된 aio는 비동기 작업을 위해 설계되었음을 나타냅니다.

DefaultAzureCredential의 목적: DefaultAzureCredential 클래스는 Azure SDK for Python의 일부입니다. 이 클래스는 Azure 서비스와 인증을 수행하는 기본적인 방법을 제공합니다. 환경 변수, 관리 ID, Azure CLI 자격 증명 등 특정 순서로 여러 방법을 시도하여 인증을 수행합니다.

비동기 작업: aio 모듈은 DefaultAzureCredential 클래스가 비동기 작업을 지원함을 나타냅니다. 이를 통해 asyncio를 사용하여 비차단 인증 요청을 수행할 수 있습니다.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# 설명:
`semantic_kernel` 패키지에서 다양한 모듈과 클래스를 가져옵니다. 각 가져오기 항목에 대한 설명은 다음과 같습니다:

`semantic_kernel.agents`에서 `AgentGroupChat`: 이 클래스는 AI 에이전트의 그룹 채팅과 관련된 기능을 처리합니다.  
`semantic_kernel.agents.azure_ai`에서 `AzureAIAgent` 및 `AzureAIAgentSettings`:

- `AzureAIAgent`: Azure AI 서비스를 활용하는 AI 에이전트를 생성하고 관리하는 데 사용됩니다.  
- `AzureAIAgentSettings`: `AzureAIAgent`의 설정을 구성하는 데 사용됩니다.  

`semantic_kernel.agents.strategies.termination.termination_strategy`에서 `TerminationStrategy`:  
이 클래스는 특정 조건에서 AI 에이전트의 실행을 종료하는 전략을 정의합니다.  

`semantic_kernel.contents.chat_message_content`에서 `ChatMessageContent`:  
이 클래스는 채팅 메시지의 내용을 처리하는 데 사용됩니다.  

`semantic_kernel.contents.utils.author_role`에서 `AuthorRole`:  
이 클래스는 채팅 메시지의 작성자와 관련된 다양한 역할을 정의합니다.  

`semantic_kernel.functions.kernel_function_decorator`에서 `kernel_function`:  
이 데코레이터는 `semantic kernel` 프레임워크 내에서 실행될 수 있는 커널 함수를 정의하는 데 사용됩니다.  

이러한 가져오기는 AI 에이전트를 생성하고 관리하며, 그룹 채팅 환경에서 상호작용할 수 있는 필수 구성 요소를 설정합니다. 예를 들어, 호텔 예약과 같은 작업을 수행하는 데 활용될 수 있습니다.  


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# 설명:
다음으로, azure.ai.projects.models 모듈에서 CodeInterpreterTool 클래스를 가져옵니다.

CodeInterpreterTool: 이 클래스는 Azure AI SDK의 일부로, AI 프로젝트의 맥락에서 코드를 해석하고 실행하는 데 사용됩니다. 이 클래스는 코드 스니펫 실행, 코드 분석, 또는 AI 워크플로우 내에서 코드 실행을 통합하는 기능을 제공합니다.  
이 가져오기는 프로젝트에서 CodeInterpreterTool을 활용하기 위한 필수 구성 요소를 설정하며, 동적으로 코드를 해석하고 실행하는 작업에 유용할 수 있습니다.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# 설명:  
ApprovalTerminationStrategy 클래스는 AI 에이전트의 작동을 종료하는 특정 전략을 제공합니다. 에이전트는 상호작용 기록의 마지막 메시지에 "saved"라는 단어가 포함되어 있으면 종료됩니다. 이는 에이전트의 작업이 "saved"라는 확인을 받으면 완료된 것으로 간주되는 상황에서 유용할 수 있습니다. 상호작용 방법을 정의하세요. 예약 계획이 저장된 후, 저장 신호를 받을 때 중지할 수 있습니다.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 설명:

이 코드 줄은 create() 메서드를 호출하여 기본 또는 사전 정의된 설정으로 AzureAIAgentSettings 객체를 초기화합니다. 이 설정 객체(ai_agent_settings)는 이후 AzureAIAgent 인스턴스를 구성하고 관리하는 데 사용될 수 있습니다.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 설명:
requests 라이브러리를 가져오면 Python 코드에서 HTTP 요청을 쉽게 수행하고 웹 서비스와 상호작용할 수 있습니다.


In [ ]:
import requests

# 설명:
이 변수는 SERP(검색 엔진 결과 페이지) API 서비스를 이용하기 위한 API 키를 저장합니다. API 키는 계정과 연관된 요청을 인증하는 데 사용되는 고유 식별자입니다.

'GOOGLE_SEARCH_API_KEY': 이 문자열은 자리 표시자입니다. 'GOOGLE_SEARCH_API_KEY'를 실제 SERP API 키로 교체해야 합니다.

목적: 이 줄의 목적은 API 키를 변수에 저장하여 SERP API 서비스에 요청을 인증하는 데 사용하는 것입니다. API 키는 서비스를 이용하고 검색을 수행하기 위해 반드시 필요합니다.

SERP API 키를 얻는 방법: SERP API 키를 얻으려면 https://serpapi.com에서 다음 일반적인 단계를 따르세요(사용 중인 특정 SERP API 서비스에 따라 정확한 단계는 다를 수 있습니다):

1. SERP API 서비스 선택:

SerpAPI, Google Custom Search JSON API 등 여러 SERP API 서비스가 있습니다. 자신의 필요에 가장 적합한 서비스를 선택하세요.

2. 계정 등록:

선택한 SERP API 서비스의 웹사이트(예: https://www.serpapi.com)로 이동하여 계정을 등록하세요. 기본 정보를 제공하고 이메일 주소를 인증해야 할 수도 있습니다.

3. API 키 생성:

등록 후 계정에 로그인하여 API 섹션 또는 대시보드로 이동하세요. 새 API 키를 생성하거나 만들 수 있는 옵션을 찾으세요.

4. API 키 복사:

API 키가 생성되면 이를 복사하세요. 이 키는 SERP API 서비스에 요청을 인증하는 데 사용됩니다.

5. 자리 표시자 교체:

.env 파일에서 자리 표시자를 실제 API 키로 교체하세요.


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 설명:
BookingPlugin 클래스는 Serpapi.com의 Google Search API를 사용하여 호텔과 항공편을 예약하는 메서드를 제공합니다. 이 클래스는 필요한 매개변수를 구성하고, API 요청을 보내며, 응답을 처리하여 관련 예약 정보를 반환합니다. API 키(SERPAPI_SEARCH_API_KEY)와 엔드포인트(SERPAPI_SEARCH_ENDPOINT)는 Google Search API에 인증하고 요청을 보내는 데 사용됩니다.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 설명:
SavePlugin 클래스는 Azure AI 서비스를 사용하여 여행 계획을 저장하는 saving_plan 메서드를 제공합니다. 이 메서드는 Azure 자격 증명을 설정하고, AI 에이전트를 생성하며, 사용자 입력을 처리하여 여행 계획 내용을 생성 및 저장하고, 파일 저장 및 정리 작업을 처리합니다. 메서드는 성공적으로 완료되면 "Saved"를 반환합니다.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 설명:
이 코드는 Azure AI 에이전트를 설정하여 사용자 입력을 기반으로 항공편 및 호텔 예약을 처리하고 여행 계획을 저장합니다. Azure 자격 증명을 사용하여 에이전트를 생성하고 구성하며, 그룹 채팅을 통해 사용자 입력을 처리하고 작업 완료 후 적절한 정리를 보장합니다. 에이전트는 각각의 작업을 수행하기 위해 특정 플러그인(BookingPlugin 및 SavePlugin)을 사용합니다.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
